In [1]:
import requests
import re
import time

import pandas as pd
import lxml
from bs4 import BeautifulSoup
from unicodedata import normalize
from geopy.geocoders import Nominatim
from haversine import haversine
#import matplotlib.pyplot as plt
import numpy as np
from openpyxl import load_workbook
import xlsxwriter
import descartes
#from shapely.geometry import Point, Polygon
# Import necessary packages
import os 
import folium
from folium import plugins
import geopandas as gpd
#import rasterio as rio
#from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et
from folium.plugins import MarkerCluster, HeatMap
from folium.plugins import FastMarkerCluster,FeatureGroupSubGroup
from folium import Popup
from flask import Flask
pd.options.display.max_rows = 4000
import IPython
from IPython.display import HTML, display
import panel as pn
import math
#import fiona
import json
from collections import Counter
import geojson
import us
import branca.colormap as cm
import datetime as dt

%matplotlib inline


C:\Users\q\anaconda3\envs\Working\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
df =  pd.read_csv("searchfund_allV7.csv")

In [3]:
df["formed"][0]

'1984-01-01'

In [4]:
#cleaning the initial df anddropping values with no search lat/long
df = df.drop(columns = ["Unnamed: 0"])
df.drop(df.loc[np.isnan(df['search lat']) ==True].index, inplace=True)
df = df.reset_index(drop = True)

In [5]:
#creating a dataframe for the acuisitions for easier anlaysis
acq_df = df[["acquisition lat", 'acquisition long', 'company','industry','acquired', 'name', 'companyWebsite', 'searchWebsite','formed','searchers','searchWebsite','investors']]
acq_df.drop(df.loc[np.isnan(df["acquisition lat"]) == True].index, inplace = True)
acq_df = acq_df.reset_index(drop = True)

C:\Users\q\anaconda3\envs\Working\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
univ_df = pd.read_csv("Colleges_and_Universities.csv")
univ_df = univ_df[["LATITUDE",'LONGITUDE','NAME','ADDRESS','CITY','STATE','ZIP','TELEPHONE','NAICS_DESC','TOT_ENROLL','WEBSITE']]
univ_df.drop(univ_df.loc[univ_df["NAICS_DESC"] == "Cosmetology and Barber Schools"].index, inplace = True)

f500_df = pd.read_csv("Fortune_500_Corporate_Headquarters.csv")
f500_df = f500_df[["LATITUDE",'LONGITUDE','NAME','ADDRESS','CITY','STATE','ZIP','COUNTY','RANK','EMPLOYEES','REVENUES','PROFIT','WEBSITE']]

ppp_county_df = pd.read_csv("PPP_loans_by_county.csv")
ppp_county_df = ppp_county_df[["COUNTY",'COUNT LOANS','SUM AVG VALUE LOANS']]

In [7]:
#trying to create easily acessible data fields for the popups
search_locations = list(zip(df["search lat"], df['search long']))
acquisition_locations = list(zip(acq_df['acquisition lat'], acq_df['acquisition long']))
acquisition_companies = list(acq_df['company'])
starbucks = pd.read_csv('starbucks_csv.csv').reset_index(drop = True)
whole_foods = pd.read_csv(r'new_whole_foods.csv')
apple = pd.read_csv(r'New_Apple.csv')
universities = pd.read_csv(r'Colleges_and_Universities.csv')#################################
fortune500s = pd.read_csv(r'Fortune_500_Corporate_Headquarters.csv')#################################
starbucks_loc = list(zip(starbucks['Latitude'], starbucks['Longitude']))
whole_loc = list(zip(whole_foods['Latitude'], whole_foods['Longitude']))
apple_loc = list(zip(apple['Latitude'], apple['Longitude']))
university_loc = list(zip(univ_df['LATITUDE'],univ_df['LONGITUDE']))###########################
f500_loc = list(zip(f500_df['LATITUDE'],f500_df['LONGITUDE']))###########################

In [8]:
#creating the default map and the marker clusters
county_map = folium.Map(location=[48,-102], zoom_start = 2, title = "mapbox")
searcher_cluster = MarkerCluster(name = "Cluster of Searchers").add_to(county_map)
acquisition_cluster = MarkerCluster(name = "Cluster of Acquisitions").add_to(county_map)
universities_cluster = MarkerCluster(name = "Cluster of Universities").add_to(county_map)##########################
fortune500s_cluster = MarkerCluster(name = "Cluster of Fortune 500 Companies").add_to(county_map)##########################
#adding toggle layers to the county mapS
#folium.LayerControl().add_to(county_map)

In [9]:
acq_df

,acquisition lat,acquisition long,company,industry,acquired,name,companyWebsite,searchWebsite,formed,searchers,searchWebsite,investors
0,32.753177,-97.332746,Uniform Printing,NaN,1984-01-01,Nova Capital,NaN,NaN,1984-01-01,'Jim Southern',NaN,'Irv Grousbeck'
1,39.739236,-104.984862,"Alta Colleges, Inc.",NaN,1987-06-01,Galena Corp Harvard University - Harvard Busi...,NaN,NaN,1985-06-01,"'Jamie Turner ', 'Kirk Riedinger'",NaN,'Rob Johnson '
2,39.127886,-76.686913,Atlas Container Corporation,NaN,1988-11-01,Hanover West,http://atlascontainer.com/,NaN,1986-01-01,"'Paul Centenari ', 'Peter Centenari '",NaN,NaN
3,37.779026,-122.419906,Western Cable Systems,NaN,1991-01-01,Balkin Corp Stanford University,NaN,NaN,1991-01-01,'Jeff Stevens',NaN,NaN
4,38.106198,-122.568119,Med-Mart now known as Pacific Pulmonary Services,NaN,1993-01-01,Braden Partners,http://www.ppsc.com/,NaN,1990-01-01,"'Peter Kelly', 'Leon Winters', 'Paul White '",NaN,'Will Thorndike'
5,45.520247,-122.674195,Cameraworld.com,NaN,1994-01-01,Sverica International,NaN,sverica.com,1993-01-01,"'Sandro Mina', 'Martin Steber', 'Gunnar Bjorkl...",sverica.com,NaN
6,43.643556,-79.565633,Chapters Inc (formed from buyouts of SmithBook...,NaN,1994-01-01,Pathfinder Capital,https://www.chapters.indigo.ca/en-ca/,NaN,1993-03-01,"'Larry Stevenson ', 'Harry Yanowitz ', 'Pamela...",NaN,NaN
7,34.186619,-118.448667,"Terry Hinge & Hardware, dba Terry Industrial",NaN,1994-01-01,Amersham Corp. Harvard University - Harvard B...,NaN,NaN,1993-06-01,'Andrew Cousin',NaN,'Jim Southern'
8,51.507322,-0.127647,RSL,NaN,1994-01-01,Simon Webster Search Fund,http://rslsteeper.com/,NaN,1991-01-01,'Simon Webster',NaN,NaN
9,42.360253,-71.058291,"ADAP, Inc",NaN,1995-01-06,Falcon Capital Corp.,NaN,NaN,1993-01-06,'David Dodson',NaN,"'Jeff Stevens', 'Ashley Giesler'"


In [10]:
#first example of adding acquistion locations onto the map, issues with adding a detailed popup - won't go past 300 in visualization
#for point in range(0, len(acquisition_locations)):
#    folium.Marker(acquisition_locations[point], popup = "Acquistion:", icon=folium.Icon(color='red', prefix = 'fa', icon_color = 'white', icon = 'check')).add_to(acquisition_cluster)
for i in range(0, len(acquisition_locations)):
    html=f"""
        <h1 style="font-size:20px"> Company Acquired: {acq_df.iloc[i]['company']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Industry: <b><br>{acq_df.iloc[i]['industry']}</b><br> </li>
            <li>Company Website (If Applicable): <b><br> <a href = {acq_df.iloc[i]['companyWebsite']}> {acq_df.iloc[i]['companyWebsite']} </a> </b><br></li>
            <li>Acquired By: <b><br>{acq_df.iloc[i]['name']}</b><br> </li>
            <li>Formation of Search fund: <b><br>{acq_df.iloc[i]['formed']}</b><br></li>
            <li>Date of Acquisition: <b><br>{acq_df.iloc[i]['acquired']}</b><br></li>
            <li>Searcher: <b><br>{acq_df.iloc[i]['searchers']}</b><br></li>
            <li>Searcher Website (If Applicable): <b><br> <a href = {acq_df.iloc[i]['searchWebsite']}> {acq_df.iloc[i]['searchWebsite']} </a> </b><br></li>
            <li>Investors: <b><br>{acq_df.iloc[i]['investors']}</b><br></li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2000)
    folium.Marker(acquisition_locations[i], popup = popup , icon=folium.Icon(color='red', prefix = 'fa', icon_color = 'white', icon = 'check')).add_to(acquisition_cluster)

In [11]:
#adding html to searchers
# add marker one by one on marker cluster
for i in range(0, len(search_locations)):
    html=f"""
        <h1 style="font-size:20px"> Search Fund:{df.iloc[i]['name']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Founded: <b><br>{df.iloc[i]['formed']}</b><br> </li>
            <li>Stage: <b><br>{df.iloc[i]['stage']}</b><br></li>
            <li>Company Acquired(If Applicable): <b><br>{df.iloc[i]['company']}</b><br></li>
            <li>Date of Acquisition: <b><br>{df.iloc[i]['acquired']}</b><br></li>
            <li>Length of Search (In Days): <b><br>{df.iloc[i]['Search Length']}</b><br></li>
            <li>Search Fund Website (If Applicable): <b><br> <a href = {df.iloc[i]['searchWebsite']}> {df.iloc[i]['searchWebsite']} </a> </b><br></li>
            <li>Company Acquired Website (If Applicable): <b><br> <a href = {df.iloc[i]['companyWebsite']}> {df.iloc[i]['companyWebsite']} </a> </b><br></li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2000)
    folium.Marker(
        location=search_locations[i],
        popup=popup,
        icon=folium.Icon(color='blue', prefix = 'fa', icon_color = 'white', icon = 'search')
    ).add_to(searcher_cluster)

In [ ]:
for i in range(0,len(fortune500s)):##################################
    html=f"""
        <h1 style="font-size:20px"> Company: {f500_df.iloc[i]['NAME']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Rank: <b><br>{f500_df.iloc[i]['RANK']}</b><br> </li>
            <li>Address: <b><br>{f500_df.iloc[i]['ADDRESS']}</b><br></li>
            <li>City: <b><br>{f500_df.iloc[i]['CITY']}</b><br> </li>
            <li>State: <b><br>{f500_df.iloc[i]['STATE']}</b><br></li>
            <li>Zip: <b><br>{f500_df.iloc[i]['ZIP']}</b><br> </li>
            <li>Employees: <b><br>{f500_df.iloc[i]['EMPLOYEES']}</b><br> </li>
            <li>Company Website (If Applicable): <b><br> <a href = {f500_df.iloc[i]['WEBSITE']}> {f500_df.iloc[i]['WEBSITE']} </a> </b><br></li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2000)
    folium.Marker(f500_loc[i], popup = popup , icon=folium.Icon(color='beige', prefix = 'fa', icon_color = 'black', icon = 'building')).add_to(fortune500s_cluster)

In [ ]:
for i in range(0,len(univ_df)):##################################
    html=f"""
        <h1 style="font-size:20px"> University: {univ_df.iloc[i]['NAME']}</h1>
        <p>Characteristics:</p>
        <ul>
            <li>Description: <b><br>{univ_df.iloc[i]['NAICS_DESC']}</b><br> </li>
            <li>Total Enrollment: <b><br>{univ_df.iloc[i]['TOT_ENROLL']}</b><br></li>
            <li>City: <b><br>{univ_df.iloc[i]['CITY']}</b><br> </li>
            <li>State: <b><br>{univ_df.iloc[i]['STATE']}</b><br></li>
            <li>University Website (If Applicable): <b><br> <a href = {univ_df.iloc[i]['WEBSITE']}> {univ_df.iloc[i]['WEBSITE']} </a> </b><br></li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2000)
    folium.Marker(university_loc[i], popup = popup , icon=folium.Icon(color='gray', prefix = 'fa', icon_color = 'white', icon = 'graduation-cap')).add_to(universities_cluster)

In [ ]:
#loading in the counties with locations
cs_df = pd.read_csv("county_location.csv")
county_est_pop = pd.read_csv("PopulationEstimates.csv",encoding='mbcs')
county_crime = pd.read_csv("crime_data_w_population_and_crime_rate.csv")
edu_df = pd.read_csv("Education.csv",encoding='mbcs')
unemployment_df = pd.read_csv("Unemployment.csv",encoding='mbcs')

fips = us.states.mapping('fips', 'name')
fips_2 = us.states.mapping('name', 'fips')
#creating a geodataframe for all US Counties
gdf = gpd.read_file('counties.geojson')
gdf = gdf.rename(columns = {'NAME':'County'})

#Dropping all search funds not in the united states
cs_df.drop(cs_df.loc[cs_df['Country']!='United States'].index, inplace=True)
cs_df = cs_df.dropna()
#cleaning data so it can be merged
cs_df["County"] = cs_df['County'].str.replace(' County','')
county_est_pop["Area_Name"] = county_est_pop['Area_Name'].str.replace(' County','')
county_crime["county_name"] = county_crime['county_name'].str.replace(' County','')
edu_df["Area name"] = edu_df['Area name'].str.replace(' County','')
unemployment_df["Area_name"] = unemployment_df['Area_name'].str.replace(' County','')
#mapping state codes onto counties so overlap is accounted for
cs_df['STATEFP'] = cs_df['State'].map(fips_2)

#grouping the variables to count the amount of searchers in each stage
stage = cs_df.groupby(['Stage','County', 'STATEFP'], as_index = False).size().pivot(["County", "STATEFP"], ["Stage"]).fillna(0)
stage = stage.reset_index()

#renaming county crime fields
county_crime = county_crime.rename(columns={'FIPS_ST':'STATEFP','FIPS_CTY':'COUNTYFP'})
county_est_pop = county_est_pop.rename(columns={'FIPStxt':'GEOID'})
edu_df = edu_df.rename(columns={'FIPS Code':'GEOID'})
unemployment_df = unemployment_df.rename(columns={'FIPS_Code':'GEOID'})
ppp_county_df = ppp_county_df.rename(columns={'COUNTY':'GEOID'})

county_crime['STATEFP']=county_crime['STATEFP'].astype(pd.StringDtype()).str.zfill(2)
county_crime['COUNTYFP']=county_crime['COUNTYFP'].astype(pd.StringDtype()).str.zfill(3)
edu_df['GEOID']=edu_df['GEOID'].astype(pd.StringDtype()).str.zfill(5)
unemployment_df['GEOID']=unemployment_df['GEOID'].astype(pd.StringDtype()).str.zfill(5)
county_est_pop['GEOID']=county_est_pop['GEOID'].astype(pd.StringDtype()).str.zfill(5)
ppp_county_df['GEOID']=ppp_county_df['GEOID'].astype(pd.StringDtype()).str.zfill(5)

#merging the GDF and County information to one large dataframe
stage_df = pd.merge(gdf, stage,  how='left', left_on=['STATEFP','County'], right_on = ['STATEFP','County']).fillna(0)
stage_df = stage_df.rename(columns={('size', 'Acquired'): "Acquired", ('size', 'Explore'): "Explore", ('size', 'Inactive'): "Inactive", ('size', 'Raise'): 'Raise', ('size', 'Search'): 'Search'})
stage_df = pd.merge(stage_df, county_crime,  how='left', left_on=['STATEFP','COUNTYFP'], right_on = ['STATEFP','COUNTYFP']).fillna(0)
stage_df = pd.merge(stage_df, county_est_pop,  how='left', left_on=['GEOID'], right_on = ['GEOID']).fillna(0)
stage_df = pd.merge(stage_df, edu_df,  how='left', left_on=['GEOID'], right_on = ['GEOID']).fillna(0)
stage_df = pd.merge(stage_df, ppp_county_df,  how='left', left_on=['GEOID'], right_on = ['GEOID']).fillna(0)

stage_df["NET_MIG_2019"]=stage_df["NET_MIG_2019"].str.replace(',','',regex=True)
stage_df["NET_MIG_2019"]=pd.to_numeric(stage_df["NET_MIG_2019"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["NET_MIG_2019"]) == True].index, inplace = True)  
stage_df["NET_MIG_2018"]=stage_df["NET_MIG_2018"].str.replace(',','',regex=True)
stage_df["NET_MIG_2018"]=pd.to_numeric(stage_df["NET_MIG_2018"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["NET_MIG_2018"]) == True].index, inplace = True)   
stage_df["NET_MIG_2017"]=stage_df["NET_MIG_2017"].str.replace(',','',regex=True)
stage_df["NET_MIG_2017"]=pd.to_numeric(stage_df["NET_MIG_2017"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["NET_MIG_2017"]) == True].index, inplace = True) 
stage_df["POP_ESTIMATE_2016"]=stage_df["POP_ESTIMATE_2016"].str.replace(',','',regex=True)
stage_df["POP_ESTIMATE_2016"]=pd.to_numeric(stage_df["POP_ESTIMATE_2016"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["POP_ESTIMATE_2016"]) == True].index, inplace = True) 
stage_df["POP_ESTIMATE_2018"]=stage_df["POP_ESTIMATE_2018"].str.replace(',','',regex=True)
stage_df["POP_ESTIMATE_2018"]=pd.to_numeric(stage_df["POP_ESTIMATE_2018"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["POP_ESTIMATE_2018"]) == True].index, inplace = True) 
stage_df["POP_ESTIMATE_2017"]=stage_df["POP_ESTIMATE_2017"].str.replace(',','',regex=True)
stage_df["POP_ESTIMATE_2017"]=pd.to_numeric(stage_df["POP_ESTIMATE_2017"])
stage_df.drop(stage_df.loc[pd.isnull(stage_df["POP_ESTIMATE_2017"]) == True].index, inplace = True) 
stage_df["AVG_MIG_RATE_16-19"]=((stage_df["NET_MIG_2019"]/stage_df["POP_ESTIMATE_2018"])+(stage_df["NET_MIG_2018"]/stage_df["POP_ESTIMATE_2017"])+(stage_df["NET_MIG_2017"]/stage_df["POP_ESTIMATE_2016"]))/3*100

#stage_df = pd.merge(stage_df, unemployment_df,  how='left', left_on=['GEOID'], right_on = ['GEOID']).fillna(0)
#turning the dataframe into a json so it can be read by the map
county_json = stage_df.to_json()

In [ ]:
county_colormap = cm.LinearColormap(["darkblue","blue","cyan"], vmin = 1, vmax = stage_df["Search"].max())
county_colormap.caption = "Searchers Per County"
style_function = lambda x: {"weight":0.5, 
                            'color': "black",
                            'fillColor':county_colormap(x['properties']['Search']) if 0 < x['properties']['Search'] else '#00000000',
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
county=folium.features.GeoJson(
        data = county_json,
        style_function=style_function,
        name = "County Hover Tool",
        control=True,
        overlay = True,
        show = False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['County','Search', 'Explore', 'Raise', 'Acquired', 'Inactive','POP_ESTIMATE_2019','crime_rate_per_100000',"Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19"],
            aliases=['County: ','Currently Searching: ', 'Currently Exploring: ', 'Currently Raising: ', 'Acquired: ', 'Inactive: ','Population Est 2019: ','Crime Rate per 100000: ',"Some college or associate's degree, 2015-19: ","Bachelor's degree or higher, 2015-19 :"],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
county_colormap.add_to(county_map)
county_map.add_child(county)

In [ ]:
loans_colormap = cm.LinearColormap(["purple","blue","cyan"], vmin = stage_df["COUNT LOANS"].astype(int).min(), vmax = stage_df["COUNT LOANS"].astype(int).max())
loans_colormap.caption = "PPP Loans by County"
style_function = lambda x: {"weight":0.5, 
                            'color': "black",
                            'fillColor':loans_colormap(x['properties']['COUNT LOANS']) ,
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
county=folium.features.GeoJson(
        data = county_json,
        style_function=style_function,
        name = "PPP Loans by County",
        control=True,
        overlay = True,
        show = False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['County','COUNT LOANS','SUM AVG VALUE LOANS'],
            aliases=['County: ','Loan Count','Estimated Total Loan Value'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
loans_colormap.add_to(county_map)
county_map.add_child(county)

In [ ]:

migration_colormap = cm.LinearColormap(["black","red","orange", "yellow","white"], vmin = stage_df["AVG_MIG_RATE_16-19"].astype(int).min(), vmax = stage_df["AVG_MIG_RATE_16-19"].astype(int).max())
migration_colormap.caption = "Average Migration Rate"
style_function = lambda x: {"weight":0.5, 
                            'color': "black",
                            'fillColor':migration_colormap(x['properties']['AVG_MIG_RATE_16-19']) ,
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
county=folium.features.GeoJson(
        data = county_json,
        style_function=style_function,
        name = "Average Migration Rate 2016-2019",
        control=True,
        overlay = True,
        show = False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['County','AVG_MIG_RATE_16-19'],
            aliases=['County: ','Average Migration Rate 2016-2019'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
migration_colormap.add_to(county_map)
county_map.add_child(county)

In [ ]:
#adding cluster groups for starbucks, apple stores, and whole foods
cluster_1 = MarkerCluster(name = "Cluster of Starbucks").add_to(county_map)
cluster_2 = MarkerCluster(name = "Cluster of Apple Stores").add_to(county_map)
cluster_3 = MarkerCluster(name = "Cluster of Whole Foods").add_to(county_map)
#adding toggle layers to the county mapS
#adding markers for all starbucks, apple stores, and whole foods in the US
for point in range(0, len(starbucks_loc)):
    try:
        folium.Marker(starbucks_loc[point], popup = "<br> <b>Starbucks</b> <br>" , icon=folium.Icon(icon = "coffee", color = 'pink', icon_color='white', prefix = 'fa'), overlay = True).add_to(cluster_1)
    except KeyError:
        print("KeyError")
for point in range(0, len(apple_loc)):
    try:
        folium.Marker(apple_loc[point], popup = "<br> <b>Apple Store</b> <br>" , icon=folium.Icon(icon = "apple", color = 'black', icon_color='white', prefix = 'fa')).add_to(cluster_2)
    except KeyError:
        print("KeyError")
for point in range(0, len(whole_loc)):
    try:
        folium.Marker(whole_loc[point], popup = "<br> <b>Whole Foods</b> <br>" , icon=folium.Icon(icon = "shopping-bag", color = 'lightgreen', icon_color='white', prefix = 'fa')).add_to(cluster_3)
    except KeyError:
        print("KeyError")

In [ ]:
#MSA CHOROPLETH
#Reading in MSA DATA
geo_data = gpd.read_file('tl_2019_us_cbsa.json')
cities = pd.read_excel('CitiesRankings.xlsx')
cities['MSA'] = cities['MSA'].str.replace(" MSA", "")
cities = cities.rename(columns={"MSA": "NAME"})
cities_set = set(cities["NAME"])
data_set = set(geo_data['NAME'])
#finding mismatch MSA's
c = list(cities_set.difference(data_set))

In [ ]:
#final_df = pd.merge(geo_data, cities, on = "NAME", how = 'outer')
#final_df = final_df.fillna(0)


In [ ]:
#final_df["geometry"] = final_df["geometry"].simplify(tolerance = 0.7)


In [ ]:
#final_df = pd.merge(geo_data, cities, on = "NAME", how = 'outer')
#final_df["Total Ranking"] = final_df['Total Ranking'].fillna(0)
#final_df["geometry"] = final_df["geometry"].simplify(tolerance = 0.7)
#geometry = final_df["geometry"].simplify(tolerance = 0.7)

In [ ]:
#MSA Choropleth
#finding the index in the cities df of all mismatched MSA's
city_index = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city = cities.loc[cities["NAME"].str.contains(i, case = False)]
    city_index.append(city["NAME"].index[0])
#finding the proper names of the MSA's given by the US Census Data
city_names = []
for i in ["Buffalo", "Detroit", "Hilton", "Miami", "Nashville", "New York", "Scranton"]:
    city_name = geo_data.loc[geo_data["NAME"].str.contains(i, case = False)]
    #print(city_name["NAME"].index[0])
    city_name = city_name.iloc[0].NAME
    city_names.append(city_name)
#replacing all values in the cities df with the correct name
count = 0
for i in city_index:
    cities.at[i, "NAME"] = city_names[count]
    count += 1


In [ ]:
final_df = pd.merge(geo_data, cities, on = "NAME", how = 'outer')
final_df["Total Ranking"] = final_df['Total Ranking'].fillna(0)
#final_df = final_df.simplify(tolerance = 0.1)
#final_df["geometry"] = final_df["geometry"].simplify(tolerance = 0.5)

In [ ]:
final_df["geometry"] = final_df["geometry"].simplify(tolerance = 0.9)

In [ ]:
msa_json = final_df.to_json()

In [ ]:
#finding the center of each MSA in the final_df and then making it into a tuple to use as a point
centers = list(zip(list(final_df["geometry"].centroid.y), list(final_df["geometry"].centroid.x)))
#adding a star for every MSA that is in the top 25
msa_cluster = MarkerCluster(name = "Top 25 MSAs").add_to(county_map)
for point in range(0, len(centers)):
    if 0 <final_df.iloc[point]["Total Ranking"] <= 25.0:
        html=f"""
            <h1 style="font-size:20px"> MSA Name: {final_df.iloc[point]['NAME']}</h1>
            <p>Characteristics:</p>
            <ul>
                <li>Total Ranking: <b><br>{final_df.iloc[point]['Total Ranking']} / 357 </b><br></li>
                <li>Ease of Business Ranking: <b><br>{final_df.iloc[point]['Ease of Business ']}</b><br></li>
                <li>Growth Ranking: <b><br>{final_df.iloc[point]['Growth']}</b><br></li>
                <li>Size Ranking: <b><br>{final_df.iloc[point]['Size']}</b><br></li>
            </ul>
            """
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2000)
        folium.Marker(
            location=centers[point],
            popup=popup,
            icon=folium.Icon(color='orange', prefix = 'fa', icon_color = 'white', icon = 'star')
        ).add_to(msa_cluster)

In [ ]:
colormap = cm.LinearColormap(["green", "yellow", "orange", "red"], vmin = 1, vmax = final_df["Total Ranking"].max())
colormap.caption = "Ranking of US MSAs"
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Total Ranking']) if 0 < x['properties']['Total Ranking'] <= 357.0 else 'white',
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
msa=folium.features.GeoJson(
        data = msa_json,
        style_function=style_function,
        name = "MSA Hover Tool",
        control=True,
        overlay = True,
        show = False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NAME','Total Ranking', 'Ease of Business ', 'Size', 'Growth'],
            aliases=['Name: ','Total Ranking: ', 'Ease of Business Ranking: ', 'Size Ranking: ', 'Growth Ranking: '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
colormap.add_to(county_map)
county_map.add_child(msa)

In [ ]:
underfished = pd.read_csv('underfished.csv')
underfished_loc = list(zip(underfished['INTPTLAT'], underfished['INTPTLON']))
#adding a star for every MSA that is in the top 25
underfished_cluster = MarkerCluster(name = "Underfished Cities").add_to(county_map)
for point in range(0, len(underfished_loc)):
        html=f"""
            <h1 style="font-size:20px"> MSA Name: {underfished.iloc[point]['NAME']}</h1>
            <p>Characteristics:</p>
            <ul>
                <li>Total Ranking: <b><br>{underfished.iloc[point]['Total Ranking']} / 357  </b><br></li>
                <li>Ease of Business Ranking: <b><br>{underfished.iloc[point]['Ease of Business ']}</b><br></li>
                <li>Growth Ranking: <b><br>{underfished.iloc[point]['Growth']}</b><br></li>
                <li>Size Ranking: <b><br>{underfished.iloc[point]['Size']}</b><br></li>
            </ul>
            """
        iframe = folium.IFrame(html=html, width=200, height=200)
        popup = folium.Popup(iframe, max_width=2000)
        folium.Marker(
            location=underfished_loc[point],
            popup=popup,
            icon=folium.Icon(color='purple', prefix = 'fa', icon_color = 'white', icon = 'arrow-down')
        ).add_to(underfished_cluster)

In [ ]:
#adding a layer to control the features on the map
folium.LayerControl().add_to(county_map)
county_map

In [ ]:
#outputting the map to an html file **Make sure to change version upon update
outfp = "county_mapv7.html"
county_map.save(outfp)

In [ ]:
#sizing down the county_map

In [ ]:
#PPP = pd.read_csv("PPP_data_150k_plus.csv",encoding='mbcs')
#PPP.drop(PPP.loc[PPP['State']=="XX"].index, inplace=True)
#PPP["fullAddress"]=PPP["City"]+" "+PPP["State"]
#PPP

In [ ]:
#geolocator = Nominatim(user_agent = 'myapplication')
#count = 0
#location_lat = []
#location_lon = []
#for val in  PPP["fullAddress"]:
#    location = geolocator.geocode(val,timeout=20)
#    try:
#        location_lat.append(location.raw['lat']) 
#        location_lon.append(location.raw['lon'])
#    except:
#        location_lat.append('nan') 
#        location_lon.append('nan')

In [ ]:
#PPP["PPP long"] = location_lon
#PPP["PPP lat"] = location_lat
#PPP